In [1]:
import psycopg2 as ps
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pandas as pd
import difflib
import matchgeo
import requests
from googletrans import Translator
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from fuzzywuzzy import fuzz



In [2]:
#! pip install transliterate
#! pip install fuzzywuzzy
#! pip install difflib

# 1. INPUT 
На вход: 
- word: искомый город
- isgoogletranslator: использвание переводчика. Если переводчик не используется, используется транслитерация.
- username: юзернэйм на геонэймс для GeonamesAPI


In [41]:
word = 'Владивасток'
word_2 = 'Росов-на-Доу'
word_4 = 'Дубаии'
word_3 = "Вилики Ноагород"
username = 'sofia_bulg'
isgoogletranslator = False
isgeonamesapi = True



Функция поиска мэтчей при помощи либо библиотеки difflib либо fuzzywuzzy, лимбо через гугл-переводчик, либо через транслитерацию

In [4]:
def match_search(word, target, n, bibl, isgoogletranslator):
    results = {}
    if isgoogletranslator:
        word = matchgeo.translate_non_latin_word(word)
    else:
        word = matchgeo.transl(word)  
    if bibl == 'difflib':          
        namesdiff = matchgeo.matches(word, target , n)       
        for city in namesdiff:
            levenshtein_distance = nltk.edit_distance(word, city) 
            results[city] = {'levenshtein_distance': levenshtein_distance}
    elif bibl == 'fw':
        namesdiff = matchgeo.matchesfuzz(word, target , n)       
        for city in namesdiff:
            distance =  fuzz.ratio(word, city)
            results[city] = {'fuzz_ratio': distance}
    else:
        print('Choose "difflib" or "fw"')
    return  namesdiff, results
    

функция поиска информации по названиям городов во всех базах geonames по городу и стране. k - количество результатов поиска, я ограничила одним, иначе ищет слишком много всего.

In [5]:
def getgeonamesinfo(cities, k=1):
    for index, row in cities.iterrows():
        city_name = row['name_1']
        ad = row['AD']
        matchgeo.geonamesapiinfo(city_name, ad, username, k) 
    print()

# 2. Загрузка и обработка данных

In [6]:
DATABASE = {
    'drivername': 'postgresql', 
    'host': 'localhost',
    'port': 5433,
    'database': 'Geonames',
    'username':'postgres',
    'password':'1207',
    'query': {}
}  

engine = create_engine(URL.create(**DATABASE))

In [7]:
query = 'SELECT * FROM "Geonames".cities5000'

data = pd.read_sql_query(query, con=engine)


In [8]:
data.dropna()
data.head(10)

,id,name_1,name_2,name_3,coord_1,coord_2,AD,location
0,﻿3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,AD,Europe/Andorra
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,AD,Europe/Andorra
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,AE,Asia/Dubai
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.9432,AE,Asia/Dubai
4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,AE,Asia/Dubai
5,291696,Khawr Fakkān,Khawr Fakkan,"Fakkan,Fakkān,Khawr Fakkan,Khawr Fakkān,Khawr ...",25.33132,56.34199,AE,Asia/Dubai
6,292223,Dubai,Dubai,"DXB,Dabei,Dibai,Dibay,Doubayi,Dubae,Dubai,Duba...",25.07725,55.30927,AE,Asia/Dubai
7,292231,Dibba Al-Fujairah,Dibba Al-Fujairah,"Al-Fujairah,BYB,Dibba Al-Fujairah,dba alfjyrt,...",25.59246,56.26176,AE,Asia/Dubai
8,292239,Dibba Al-Hisn,Dibba Al-Hisn,"BYB,Daba,Daba al-Hisn,Dabā,Dabā al-Ḥiṣn,Diba,D...",25.61955,56.27291,AE,Asia/Dubai
9,292672,Sharjah,Sharjah,"Al Sharjah,Ash 'Mariqah,Ash Shariqa,Ash Shariq...",25.33737,55.41206,AE,Asia/Dubai


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26685 entries, 0 to 26684
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        26685 non-null  object
 1   name_1    26685 non-null  object
 2   name_2    26685 non-null  object
 3   name_3    24426 non-null  object
 4   coord_1   26685 non-null  object
 5   coord_2   26685 non-null  object
 6   AD        26685 non-null  object
 7   location  26685 non-null  object
dtypes: object(8)
memory usage: 1.6+ MB


In [10]:
data['name_3'].isna().sum()

2259

In [11]:
data['name_3'] = data['name_3'].fillna('unknown')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26685 entries, 0 to 26684
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        26685 non-null  object
 1   name_1    26685 non-null  object
 2   name_2    26685 non-null  object
 3   name_3    26685 non-null  object
 4   coord_1   26685 non-null  object
 5   coord_2   26685 non-null  object
 6   AD        26685 non-null  object
 7   location  26685 non-null  object
dtypes: object(8)
memory usage: 1.6+ MB


In [12]:
target = data['name_1']

# 3. OUTPUT

## 3.1 difflib

с переводчиком

In [13]:
namesdiff, vocab = match_search(word,target, 5, 'difflib', True)
vocab

{'Vladivostok': {'levenshtein_distance': 0},
 'Valdosta': {'levenshtein_distance': 6},
 'Vasto': {'levenshtein_distance': 6},
 'Gladstone': {'levenshtein_distance': 6}}

с транслитом

In [14]:
namesdiff, vocab = match_search(word,target, 5, 'difflib', False)
vocab

{'Vladivostok': {'levenshtein_distance': 1},
 'Valdivia': {'levenshtein_distance': 7},
 'Vasto': {'levenshtein_distance': 6},
 'Gladstone': {'levenshtein_distance': 6}}

в geonames

In [15]:
closest_cities_data = matchgeo.sorted_data(data, namesdiff)
closest_cities_data

C:\Users\user\Desktop\geomatchesloc\matchgeo.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnames.loc[data['name_1'].isin(names), 'name_order'] = dfnames['name_1'].apply(


,id,name_1,name_2,name_3,coord_1,coord_2,AD,location
20699,2013348,Vladivostok,Vladivostok,"Bladibostok,Uladzivastok,VVO,Vladivostok,Vladi...",43.10562,131.87353,RU,Asia/Vladivostok
3295,3868707,Valdivia,Valdivia,"Ciudad de Valdivia,Ciudad de Valdivía,Val'divi...",-39.81422,-73.24589,CL,America/Santiago
14043,3164672,Vasto,Vasto,"Istonio,Vasto,Васто",42.1115,14.70649,IT,Europe/Rome
615,2165478,Gladstone,Gladstone,"GLT,Gladston,Gladstone,Gledston,Glehdston,ge l...",-23.84852,151.25775,AU,Australia/Brisbane
23302,4387990,Gladstone,Gladstone,"Evanston,Gladston,Gladstone,Gladstoun,Gledston...",39.20389,-94.55468,US,America/Chicago


In [16]:
getgeonamesinfo(closest_cities_data)

Data for Vladivostok:
adminCode1: 59
lng: 131.87353
geonameId: 2013348
toponymName: Vladivostok
countryId: 2017370
fcl: P
population: 604901
countryCode: RU
name: Vladivostok
fclName: city, village,...
adminCodes1: {'ISO3166_2': 'PRI'}
countryName: Russia
fcodeName: seat of a first-order administrative division
adminName1: Primorye
lat: 43.10562
fcode: PPLA


Data for Valdivia:
adminCode1: 17
lng: -73.24589
geonameId: 3868707
toponymName: Valdivia
countryId: 3895114
fcl: P
population: 133419
countryCode: CL
name: Valdivia
fclName: city, village,...
adminCodes1: {'ISO3166_2': 'LR'}
countryName: Chile
fcodeName: seat of a first-order administrative division
adminName1: Los Ríos Region
lat: -39.81422
fcode: PPLA


Data for Vasto:
adminCode1: 01
lng: 14.70649
geonameId: 3164672
toponymName: Vasto
countryId: 3175395
fcl: P
population: 41409
countryCode: IT
name: Vasto
fclName: city, village,...
adminCodes1: {'ISO3166_2': '65'}
countryName: Italy
fcodeName: seat of a third-order administrati

## 3.2 fuzzywuzzy

с переводчиком

In [17]:
namesfw, vocab = match_search(word,target, 5, 'fw', True)
vocab

{'Vladivostok': {'fuzz_ratio': 100},
 'Istok': {'fuzz_ratio': 50},
 'Rostock': {'fuzz_ratio': 56},
 'Divo': {'fuzz_ratio': 40},
 'Most': {'fuzz_ratio': 40}}

с транслитом

In [18]:
namesfw, vocab = match_search(word,target, 5, 'fw', False)
vocab

{'Vladivostok': {'fuzz_ratio': 91},
 'Pasto': {'fuzz_ratio': 50},
 'Vasto': {'fuzz_ratio': 62},
 'Rivas': {'fuzz_ratio': 50},
 'Sivas': {'fuzz_ratio': 50}}

С geonames

In [19]:
closest_cities_data = matchgeo.sorted_data(data, namesdiff)
closest_cities_data

C:\Users\user\Desktop\geomatchesloc\matchgeo.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnames.loc[data['name_1'].isin(names), 'name_order'] = dfnames['name_1'].apply(


,id,name_1,name_2,name_3,coord_1,coord_2,AD,location
20699,2013348,Vladivostok,Vladivostok,"Bladibostok,Uladzivastok,VVO,Vladivostok,Vladi...",43.10562,131.87353,RU,Asia/Vladivostok
3295,3868707,Valdivia,Valdivia,"Ciudad de Valdivia,Ciudad de Valdivía,Val'divi...",-39.81422,-73.24589,CL,America/Santiago
14043,3164672,Vasto,Vasto,"Istonio,Vasto,Васто",42.1115,14.70649,IT,Europe/Rome
615,2165478,Gladstone,Gladstone,"GLT,Gladston,Gladstone,Gledston,Glehdston,ge l...",-23.84852,151.25775,AU,Australia/Brisbane
23302,4387990,Gladstone,Gladstone,"Evanston,Gladston,Gladstone,Gladstoun,Gledston...",39.20389,-94.55468,US,America/Chicago


In [20]:
getgeonamesinfo(closest_cities_data)

Data for Vladivostok:
adminCode1: 59
lng: 131.87353
geonameId: 2013348
toponymName: Vladivostok
countryId: 2017370
fcl: P
population: 604901
countryCode: RU
name: Vladivostok
fclName: city, village,...
adminCodes1: {'ISO3166_2': 'PRI'}
countryName: Russia
fcodeName: seat of a first-order administrative division
adminName1: Primorye
lat: 43.10562
fcode: PPLA


Data for Valdivia:
adminCode1: 17
lng: -73.24589
geonameId: 3868707
toponymName: Valdivia
countryId: 3895114
fcl: P
population: 133419
countryCode: CL
name: Valdivia
fclName: city, village,...
adminCodes1: {'ISO3166_2': 'LR'}
countryName: Chile
fcodeName: seat of a first-order administrative division
adminName1: Los Ríos Region
lat: -39.81422
fcode: PPLA


Data for Vasto:
adminCode1: 01
lng: 14.70649
geonameId: 3164672
toponymName: Vasto
countryId: 3175395
fcl: P
population: 41409
countryCode: IT
name: Vasto
fclName: city, village,...
adminCodes1: {'ISO3166_2': '65'}
countryName: Italy
fcodeName: seat of a third-order administrati

# 4. Пересечение множеств

In [21]:
common_values = list(set(namesdiff) & set(namesfw))
common_values

['Vasto', 'Vladivostok']

In [22]:
cities_common = matchgeo.sorted_data(data, common_values)
cities_common

C:\Users\user\Desktop\geomatchesloc\matchgeo.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnames.loc[data['name_1'].isin(names), 'name_order'] = dfnames['name_1'].apply(


,id,name_1,name_2,name_3,coord_1,coord_2,AD,location
14043,3164672,Vasto,Vasto,"Istonio,Vasto,Васто",42.1115,14.70649,IT,Europe/Rome
20699,2013348,Vladivostok,Vladivostok,"Bladibostok,Uladzivastok,VVO,Vladivostok,Vladi...",43.10562,131.87353,RU,Asia/Vladivostok


In [23]:
getgeonamesinfo(cities_common)

Data for Vasto:
adminCode1: 01
lng: 14.70649
geonameId: 3164672
toponymName: Vasto
countryId: 3175395
fcl: P
population: 41409
countryCode: IT
name: Vasto
fclName: city, village,...
adminCodes1: {'ISO3166_2': '65'}
countryName: Italy
fcodeName: seat of a third-order administrative division
adminName1: Abruzzo
lat: 42.1115
fcode: PPLA3


Data for Vladivostok:
adminCode1: 59
lng: 131.87353
geonameId: 2013348
toponymName: Vladivostok
countryId: 2017370
fcl: P
population: 604901
countryCode: RU
name: Vladivostok
fclName: city, village,...
adminCodes1: {'ISO3166_2': 'PRI'}
countryName: Russia
fcodeName: seat of a first-order administrative division
adminName1: Primorye
lat: 43.10562
fcode: PPLA





Для "Росов-на-Доу"

In [42]:
namesdiff, vocab = match_search(word_2,target, 5, 'difflib', True)
vocab

{'Rostov-na-Donu': {'levenshtein_distance': 5}}

In [43]:
namesdiff, vocab = match_search(word_2,target, 5, 'difflib', False)
vocab

{'Rostov-na-Donu': {'levenshtein_distance': 2}}

In [44]:
namesfw, vocab = match_search(word_2,target, 5, 'fw', True)
vocab

{'Rostov-na-Donu': {'fuzz_ratio': 77},
 'Rostov': {'fuzz_ratio': 56},
 'Rosso': {'fuzz_ratio': 47},
 'Yovon': {'fuzz_ratio': 47},
 'Avon': {'fuzz_ratio': 38}}

In [45]:
namesfw, vocab = match_search(word_2,target, 5, 'fw', True)
vocab

{'Rostov-na-Donu': {'fuzz_ratio': 77},
 'Rostov': {'fuzz_ratio': 56},
 'Rosso': {'fuzz_ratio': 47},
 'Yovon': {'fuzz_ratio': 47},
 'Avon': {'fuzz_ratio': 38}}

In [28]:
common_values = list(set(namesdiff) & set(namesfw))
common_values

['Rostov-na-Donu']

Для "Вилики Ноагород"

In [29]:
namesdiff, vocab = match_search(word_3,target, 5, 'difflib', True)
vocab

{'Velikiy Novgorod': {'levenshtein_distance': 0},
 'Nizhniy Novgorod': {'levenshtein_distance': 5},
 'Veliko Tŭrnovo': {'levenshtein_distance': 8},
 'Velikiy Ustyug': {'levenshtein_distance': 8}}

In [30]:
namesdiff, vocab = match_search(word_3,target, 5, 'difflib', False)
vocab

{'Velikiy Novgorod': {'levenshtein_distance': 3},
 'Nizhniy Novgorod': {'levenshtein_distance': 6}}

In [31]:
namesfw, vocab = match_search(word_3,target, 5, 'fw', True)
vocab

{'Velikiy Novgorod': {'fuzz_ratio': 100},
 'Ngoro': {'fuzz_ratio': 48},
 'Bogor': {'fuzz_ratio': 38},
 'Bogo': {'fuzz_ratio': 30},
 'Verl': {'fuzz_ratio': 30}}

In [32]:
namesfw, vocab = match_search(word_3,target, 5, 'fw', False)
vocab

{'Kagoro': {'fuzz_ratio': 48},
 'Ngoro': {'fuzz_ratio': 50},
 'Bogor': {'fuzz_ratio': 40},
 'Velikiy Novgorod': {'fuzz_ratio': 84},
 'Nola': {'fuzz_ratio': 32}}

In [33]:
common_values = list(set(namesdiff) & set(namesfw))
common_values

['Velikiy Novgorod']

Для Дубаии

In [34]:
namesdiff, vocab = match_search(word_4,target, 5, 'difflib', True)
vocab

{'Dubai': {'levenshtein_distance': 1},
 'Duba': {'levenshtein_distance': 2},
 'Duoba': {'levenshtein_distance': 3},
 'Dumai': {'levenshtein_distance': 2},
 'Dubti': {'levenshtein_distance': 2}}

In [35]:
namesdiff, vocab = match_search(word_4,target, 5, 'difflib', False)
vocab

{'Dubai': {'levenshtein_distance': 1},
 'Duba': {'levenshtein_distance': 2},
 'Duoba': {'levenshtein_distance': 3},
 'Dumai': {'levenshtein_distance': 2},
 'Dubti': {'levenshtein_distance': 2}}

In [36]:
namesfw, vocab = match_search(word_4,target, 5, 'fw', True)
vocab

{'Dubai': {'fuzz_ratio': 91},
 'Duba': {'fuzz_ratio': 80},
 'Duoba': {'fuzz_ratio': 73},
 'Dubti': {'fuzz_ratio': 73},
 'Douai': {'fuzz_ratio': 73}}

In [37]:
namesfw, vocab = match_search(word_4,target, 5, 'fw', False)
vocab

{'Dubai': {'fuzz_ratio': 91},
 'Duba': {'fuzz_ratio': 80},
 'Duoba': {'fuzz_ratio': 73},
 'Dubti': {'fuzz_ratio': 73},
 'Douai': {'fuzz_ratio': 73}}

In [38]:
common_values = list(set(namesdiff) & set(namesfw))
common_values

['Duba', 'Dubai', 'Dubti', 'Duoba']